# Span Batch Analysis - Data Preparation

## Initialization

Before start, Fill in [secret.yml](span-batch-analysis/secret.yaml). Refer to [secret_example.yml](span-batch-analysis/secret_example.yaml). This is for specifying RPC endpoints.

Also check config located at [config](span-batch-analysis/config/).

In [ ]:
import os

Build binaries.

In [ ]:
%%bash
cd cmd/batch_decoder && go build . && cd ../../
cd cmd/span_batch_tester && go build . && cd ../../

In [ ]:
%%capture
%env BATCH_DECODER=cmd/batch_decoder/batch_decoder
%env SPAN_BATCH_TESTER=cmd/span_batch_tester/span_batch_tester

Clone superchain config.

In [ ]:
%%bash
git clone https://github.com/ethereum-optimism/superchain-registry
cd superchain-registry && git checkout 5d3ca1966435827ac8c4216bddcb541ec64c1d6c && cd --

Select your chain.

In [ ]:
%%capture
%env L1_CHAIN_NAME=mainnet
%env L2_CHAIN_NAME=op

# available networks
#      L1 - L2 list
# mainnet - base, op, pgn, zora
# goerli  - base, op, zora
# sepolia - op, pgn

In [ ]:
l1_chain_name = os.environ["L1_CHAIN_NAME"]
l2_chain_name = os.environ["L2_CHAIN_NAME"]

start_number = !yq eval ".start_l1_number" "span-batch-analysis/config/{l1_chain_name}/{l2_chain_name}.yaml"
start_number_value = int(start_number[0])
end_number = !yq eval ".end_l1_number" "span-batch-analysis/config/{l1_chain_name}/{l2_chain_name}.yaml"
end_number_value = int(end_number[0])
batch_tx_sender = !yq eval ".batch_tx_sender" "span-batch-analysis/config/{l1_chain_name}/{l2_chain_name}.yaml"
batch_tx_sender_value = batch_tx_sender[0]
l2_genesis_timestamp = !yq eval ".genesis.l2_time" "superchain-registry/superchain/configs/{l1_chain_name}/{l2_chain_name}.yaml"
l2_genesis_timestamp_value = int(l2_genesis_timestamp[0])
l2_chain_id = !yq eval ".chain_id" "superchain-registry/superchain/configs/{l1_chain_name}/{l2_chain_name}.yaml"
l2_chain_id_value = int(l2_chain_id[0])
inbox_addr = !yq eval ".batch_inbox_addr" "superchain-registry/superchain/configs/{l1_chain_name}/{l2_chain_name}.yaml"
inbox_addr_value = inbox_addr[0]

acc_start_number = !yq eval ".acc_l2_start_number" "span-batch-analysis/config/{l1_chain_name}/{l2_chain_name}.yaml"
acc_start_number_value = int(acc_start_number[0])
acc_end_number = !yq eval ".acc_l2_end_number" "span-batch-analysis/config/{l1_chain_name}/{l2_chain_name}.yaml"
acc_end_number_value = int(acc_end_number[0])

Read analyis config.

In [ ]:
%env L1_START=$start_number_value
%env L1_END=$end_number_value
%env BATCH_TX_SENDER=$batch_tx_sender_value
%env L2_GENESIS_TIMESTAMP=$l2_genesis_timestamp_value
%env L2_CHAIN_ID=$l2_chain_id_value
%env INBOX=$inbox_addr_value

%env ACC_L2_START=$acc_start_number_value
%env ACC_L2_END=$acc_end_number_value


Get L1 RPC from secret.

In [ ]:
key_name = f"L1_{l1_chain_name}_RPC".upper()
L1_RPC = !yq eval ".{key_name}" span-batch-analysis/secret.yaml
L1_RPC_ENDPOINT = L1_RPC[0]

key_name = f"L2_{l2_chain_name}_{l1_chain_name}_RPC".upper()
L2_RPC = !yq eval ".{key_name}" span-batch-analysis/secret.yaml
L2_RPC_ENDPOINT = L2_RPC[0]

In [ ]:
%%capture
%env L1_RPC=$L1_RPC_ENDPOINT
%env L2_RPC=$L2_RPC_ENDPOINT

## Data Collection

All data will be stored at `./artifacts/{L1_CHAIN_NAME}/{L2_CHAIN_NAME}`

In [ ]:
l1_chain_name = os.environ["L1_CHAIN_NAME"]
l2_chain_name = os.environ["L2_CHAIN_NAME"]

%env DB_PREFIX=./artifacts/$l1_chain_name/$l2_chain_name

batch decoder `fetch`: Read channel frames from L1 by reading batch transactions.

In [ ]:
!$BATCH_DECODER fetch --l1=$L1_RPC --start=$L1_START --end=$L1_END --inbox=$INBOX --out=$DB_PREFIX/transactions_cache --sender=$BATCH_TX_SENDER --concurrent-requests=25 | tail -10

batch decoder `reassemble`: Reassemble channel frames to channels. 

In [ ]:
!$BATCH_DECODER reassemble --inbox=$INBOX --in=$DB_PREFIX/transactions_cache --out=$DB_PREFIX/channel_cache | tail -10

span batch tester `convert`: Convert channels with singular batches to span batches. Try multiple tx encoding types.

In [ ]:
!TX_ENCODE_TYPE=0  && $SPAN_BATCH_TESTER convert --in=$DB_PREFIX/channel_cache --out=$DB_PREFIX/span_batch_cache_v$TX_ENCODE_TYPE --l2=$L2_RPC --genesis-timestamp=$L2_GENESIS_TIMESTAMP --chain-id=$L2_CHAIN_ID --tx-type=$TX_ENCODE_TYPE | tail -10

In [ ]:
!TX_ENCODE_TYPE=1  && $SPAN_BATCH_TESTER convert --in=$DB_PREFIX/channel_cache --out=$DB_PREFIX/span_batch_cache_v$TX_ENCODE_TYPE --l2=$L2_RPC --genesis-timestamp=$L2_GENESIS_TIMESTAMP --chain-id=$L2_CHAIN_ID --tx-type=$TX_ENCODE_TYPE | tail -10

In [ ]:
!TX_ENCODE_TYPE=2  && $SPAN_BATCH_TESTER convert --in=$DB_PREFIX/channel_cache --out=$DB_PREFIX/span_batch_cache_v$TX_ENCODE_TYPE --l2=$L2_RPC --genesis-timestamp=$L2_GENESIS_TIMESTAMP --chain-id=$L2_CHAIN_ID --tx-type=$TX_ENCODE_TYPE | tail -10

span batch tester `analyze`: Analyze channels with singluar batches by comparing with corresponding span batches.

In [ ]:
!TX_ENCODE_TYPE=0 && $SPAN_BATCH_TESTER analyze --in-channel=$DB_PREFIX/channel_cache --in-span-batch=$DB_PREFIX/span_batch_cache_v$TX_ENCODE_TYPE --out=$DB_PREFIX/result_v$TX_ENCODE_TYPE --chain-id=$L2_CHAIN_ID --tx-type=$TX_ENCODE_TYPE | tail -10

In [ ]:
!TX_ENCODE_TYPE=1 && $SPAN_BATCH_TESTER analyze --in-channel=$DB_PREFIX/channel_cache --in-span-batch=$DB_PREFIX/span_batch_cache_v$TX_ENCODE_TYPE --out=$DB_PREFIX/result_v$TX_ENCODE_TYPE --chain-id=$L2_CHAIN_ID --tx-type=$TX_ENCODE_TYPE | tail -10

In [ ]:
!TX_ENCODE_TYPE=2 && $SPAN_BATCH_TESTER analyze --in-channel=$DB_PREFIX/channel_cache --in-span-batch=$DB_PREFIX/span_batch_cache_v$TX_ENCODE_TYPE --out=$DB_PREFIX/result_v$TX_ENCODE_TYPE --chain-id=$L2_CHAIN_ID --tx-type=$TX_ENCODE_TYPE | tail -10

span batch tester `fetch`: Fetch consecutive singular batches.

In [ ]:
!$SPAN_BATCH_TESTER fetch --l2 $L2_RPC --start=$ACC_L2_START --end=$ACC_L2_END --concurrent-requests=100 --out=$DB_PREFIX/singular_batches_cache | tail -10

span batch tester `merge`: Merge singular batches:

This command will analyze `start - end` number of batches, Comparing uncompressed size of below two batches:
- For `i` in range  `0` to `start - end - 1`
  - singular batch from number `start` to `start + i`, inclusive.
  - span batch from number `start` to `start + i`, inclusive.

Upper command will make a result json file which has name of the form `{start_number}_{end_number}.json`.

In [ ]:
!$SPAN_BATCH_TESTER merge --start=$ACC_L2_START --end=$ACC_L2_END --l2 $L2_RPC --genesis-timestamp=$L2_GENESIS_TIMESTAMP --in=$DB_PREFIX/singular_batches_cache --out=$DB_PREFIX/merge_result | tail -10

span batch tester `format`: Compare different tx encodings for span batch.


In [ ]:
!$SPAN_BATCH_TESTER format --permutation 0123465 --in-span-batch=$DB_PREFIX/span_batch_cache_v2 --out=$DB_PREFIX/format_result --chain-id=$L2_CHAIN_ID | tail -10

In [ ]:
!$SPAN_BATCH_TESTER format --permutation 0134625 --in-span-batch=$DB_PREFIX/span_batch_cache_v2 --out=$DB_PREFIX/format_result --chain-id=$L2_CHAIN_ID | tail -10

In [ ]:
!$SPAN_BATCH_TESTER format --permutation 3041265 --in-span-batch=$DB_PREFIX/span_batch_cache_v2 --out=$DB_PREFIX/format_result --chain-id=$L2_CHAIN_ID | tail -10

In [ ]:
!$SPAN_BATCH_TESTER format --permutation 0125634 --in-span-batch=$DB_PREFIX/span_batch_cache_v2 --out=$DB_PREFIX/format_result --chain-id=$L2_CHAIN_ID | tail -10

In [ ]:
!$SPAN_BATCH_TESTER format --permutation 0324165 --in-span-batch=$DB_PREFIX/span_batch_cache_v2 --out=$DB_PREFIX/format_result --chain-id=$L2_CHAIN_ID | tail -10

In [ ]:
!$SPAN_BATCH_TESTER format --permutation 3041256 --in-span-batch=$DB_PREFIX/span_batch_cache_v2 --out=$DB_PREFIX/format_result --chain-id=$L2_CHAIN_ID | tail -10

In [ ]:
!$SPAN_BATCH_TESTER format --permutation 3401256 --in-span-batch=$DB_PREFIX/span_batch_cache_v2 --out=$DB_PREFIX/format_result --chain-id=$L2_CHAIN_ID | tail -10

In [ ]:
!$SPAN_BATCH_TESTER format --permutation 6051234 --in-span-batch=$DB_PREFIX/span_batch_cache_v2 --out=$DB_PREFIX/format_result --chain-id=$L2_CHAIN_ID | tail -10

In [ ]:
!$SPAN_BATCH_TESTER format --permutation 0132456 --in-span-batch=$DB_PREFIX/span_batch_cache_v2 --out=$DB_PREFIX/format_result --chain-id=$L2_CHAIN_ID | tail -10

In [ ]:
!$SPAN_BATCH_TESTER format --permutation 0352614 --in-span-batch=$DB_PREFIX/span_batch_cache_v2 --out=$DB_PREFIX/format_result --chain-id=$L2_CHAIN_ID | tail -10